In [1]:
from datetime import datetime
from pprint import pprint
from time import sleep
import csv
import locale
import numpy as np
import pandas as pd
import requests

In [2]:
#Creando Dataframe para cargarlo 

archivo = "datos_airtable.csv"

df_descargado = pd.read_csv(archivo)

In [3]:
df_descargado.head(2)

,id,createdTime,fields.Temp Max,fields.Viento,fields.Lluvia,fields.Tiempo,fields.Temp Min,fields.Ubicacion,fields.Fecha
0,rec01K8KaBvXI5EDf,2024-01-10T18:09:14.000Z,24,18,0.0,Parcialmente nublado,12,Madrid,18 de abril 2022
1,rec01RliefQAF5CmE,2024-01-10T18:05:28.000Z,25,14,0.0,Nublado,17,Málaga,11 de mayo 2023


In [2]:
def cargar_datos(df):
    
    #Detalles de autenticación de Airtable
    api_key = 'patQcUpVqSdKlfmfr.56b544d0dcb62db4f4d5020e47e45b95605dc080ab4223d8d8aac9cf778ca346'
    base_id = 'appw8XFmZCmNnf4eL'
    table_name = 'tblOgjJormJcrh8sT'

    #URL
    url = f'https://api.airtable.com/v0/{base_id}/{table_name}'

    #Convierte el dataFrame a un formato que se pueda enviar
    datos_json = [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])]

    #Encabezado
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }


    for i in range(0, df.shape[0], 10):

        data = {"records" : datos_json[i : i + 10]}

        response = requests.post(url, json = data, headers = headers)

        sleep(0.6)

        if response.status_code == 200:
            print("Datos cargados en Airtable.")
        else:
            print(f"Error al cargar datos: {response.text}")

In [9]:
def extract_datos():

    #Detalles de autenticación de Airtable
    api_key = 'patQcUpVqSdKlfmfr.56b544d0dcb62db4f4d5020e47e45b95605dc080ab4223d8d8aac9cf778ca346'
    base_id = 'appw8XFmZCmNnf4eL'
    table_name = 'tblOgjJormJcrh8sT'

    #URL
    url = f'https://api.airtable.com/v0/{base_id}/{table_name}'

    #Encabezado
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    #Parametro de salida o exit
    params = {"offset" : None}

    df_descargado = pd.DataFrame()

    while params.get("offset") != None or df_descargado.shape[0] == 0:

        response = requests.get(url, headers = headers, params = params)

        if response.status_code == 200:
            print("Descarga completada")

        params["offset"] = response.json().get("offset")

        print(params.get("offset"))

        df_descargado = pd.concat([df_descargado, pd.json_normalize(response.json()["records"])], ignore_index = True)

        sleep(0.6)
        
    return df_descargado

In [ ]:
#Esto devuelve un dataframe
df_descargado = extract_datos()

In [12]:
#Hemos adjuntado el archivo descargado de Airtable en formato csv para que se pueda visualizar.

df_descargado.to_csv("datos_airtable.csv", index = False)